# В этом ноутбуке сравниваем усредненные стоимости путей для CBS и субоптимального ECBS

In [1]:
from stats import run_algo
from typing import List
from tqdm.notebook import tqdm
import matplotlib
import numpy as np
import time
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

In [2]:
import os
filename = 'result_ecbs_cbs_comp.txt'

from collections import defaultdict
def parse_path_file(path_file_content):
    agents_path = defaultdict(list)
    agent = None
    lines = path_file_content.split('\n')[1:]
    for line in lines:
        if line == '':
            break
        nums = list(map(int, line.split(' ')))
        if len(nums) == 1:
            agent = int(nums[0])
        else:
            x, y, time = nums
            agents_path[agent].append((x, y, time))
    return agents_path

def read_paths(filename=filename):
    with open(filename, 'r') as path_file:
        path_file_content = path_file.read()
        agents_path = parse_path_file(path_file_content)
    return dict(agents_path)

def count_sol_cost(agents_path):
    cost = 0
    for path in agents_path.values():
        cost += path[-1][2]
    return cost

def run(alg_name, map_name, scen_name, num_agents, num_repetitions, w=None, timeout=2*60):
    costs = []
    times = []
    count_done = 0
    for i in tqdm(range(num_repetitions)):
        start = time.time()
        done = run_algo(alg=alg_name, dest_file=filename,
                        map_path=os.path.join(os.curdir, 'data', 'maps', 'mapf', map_name),
                        scen_path=os.path.join(os.curdir, 'data', 'scens', 'mapf', scen_name),
                        tasks_count=num_agents,
                        test_num=i, w=w, timeout=timeout)
        times.append(time.time() - start)
        if done:
            costs.append(count_sol_cost(read_paths()))
            count_done += int(done)
        else:
            costs.append(None)
    if count_done == 0:
        return (0, 0, 0)
    return count_done / num_repetitions, costs, times

In [3]:
def gen_cmp_list(map_name:str, scen_name:str, actors_array:List[int], N=30, w=None):
    rate_list = []
    for num_actors in actors_array:
        rate, costs, times = run('ECBS', map_name, scen_name, num_actors, N, w)
        rate_, costs_, times_ = run('CBS', map_name, scen_name, num_actors, N)
        avg_cost, avg_cost_ = 0, 0
        avg_t, avg_t_ = 0, 0
        done = 0
        for i, (cost, cost_) in enumerate(zip(costs, costs_)):
            if cost is not None and cost_ is not None:
                avg_cost += cost
                avg_cost_ += cost_
                avg_t += times[i]
                avg_t_ += times_[i]
                done += 1
        avg_cost /= done
        avg_cost_ /= done
        avg_t /= done
        avg_t_ /= done

        print(f'ECBS - Num actors: {num_actors}, Rate: {rate:.2f}, avg cost: {avg_cost:.2f}, time: {avg_t: .2f}')
        print(f'CBS - Num actors: {num_actors}, Rate: {rate_:.2f}, avg cost: {avg_cost_:.2f}, time: {avg_t_: .2f}')
        print(f'Passed in common: {done / len(costs)}, diff: {avg_cost / avg_cost_: .4f}')

        rate_list.append((rate, rate_, avg_cost, avg_cost_, avg_t, avg_t_))
        if rate <= 0.1:
            break
    while len(rate_list) < len (actors_array):
        rate_list.append((0.0, 0, 0))
    
    return rate_list

In [4]:
rand_actors = [5, 7, 10, 13, 16, 20, 23, 26]

In [5]:
# CBS vs ECBS 1.5 random-32x32x10
rand = gen_cmp_list('random-32-32-10.map', 'random-32-32-10-even-1.scen', rand_actors, w=1.5)
rand

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

ECBS - Num actors: 5, Rate: 1.00, avg cost: 105.63, time:  0.01
CBS - Num actors: 5, Rate: 1.00, avg cost: 105.63, time:  0.06
Passed in common: 1.0, diff:  1.0000


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

ECBS - Num actors: 23, Rate: 1.00, avg cost: 480.97, time:  0.09
CBS - Num actors: 23, Rate: 1.00, avg cost: 480.50, time:  0.44
Passed in common: 1.0, diff:  1.0010


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

ECBS - Num actors: 26, Rate: 1.00, avg cost: 540.29, time:  0.10
CBS - Num actors: 26, Rate: 0.93, avg cost: 539.71, time:  1.38
Passed in common: 0.9333333333333333, diff:  1.0011


[(1.0,
  1.0,
  105.63333333333334,
  105.63333333333334,
  0.013932538032531739,
  0.056404741605122884),
 (1.0, 1.0, 148.9, 148.8, 0.019509402910868327, 0.059574826558430986),
 (1.0,
  1.0,
  213.73333333333332,
  213.6,
  0.028514711062113444,
  0.09523428281148275),
 (1.0,
  1.0,
  272.6666666666667,
  272.43333333333334,
  0.03243541717529297,
  0.17685987949371337),
 (1.0,
  1.0,
  336.96666666666664,
  336.6666666666667,
  0.049280953407287595,
  0.19191856384277345),
 (1.0,
  1.0,
  421.23333333333335,
  420.8666666666667,
  0.08819425900777181,
  0.3257248322168986),
 (1.0, 1.0, 480.96666666666664, 480.5, 0.08602189222971598, 0.444214924176534),
 (1.0,
  0.9333333333333333,
  540.2857142857143,
  539.7142857142857,
  0.09969860315322876,
  1.381485947540828)]

In [6]:
# CBS vs ECBS 1.3 random-32x32x10
rand_2 = gen_cmp_list('random-32-32-10.map', 'random-32-32-10-even-1.scen', rand_actors, w=1.3)
rand_2

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

ECBS - Num actors: 5, Rate: 1.00, avg cost: 105.63, time:  0.01
CBS - Num actors: 5, Rate: 1.00, avg cost: 105.63, time:  0.06
Passed in common: 1.0, diff:  1.0000


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

ECBS - Num actors: 23, Rate: 1.00, avg cost: 480.97, time:  0.07
CBS - Num actors: 23, Rate: 1.00, avg cost: 480.50, time:  0.44
Passed in common: 1.0, diff:  1.0010


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

ECBS - Num actors: 26, Rate: 1.00, avg cost: 540.29, time:  0.07
CBS - Num actors: 26, Rate: 0.93, avg cost: 539.71, time:  1.40
Passed in common: 0.9333333333333333, diff:  1.0011


[(1.0,
  1.0,
  105.63333333333334,
  105.63333333333334,
  0.01318217913309733,
  0.05801082452138265),
 (1.0, 1.0, 148.9, 148.8, 0.017759339014689127, 0.060416158040364584),
 (1.0,
  1.0,
  213.73333333333332,
  213.6,
  0.024018041292826333,
  0.09910641511281332),
 (1.0,
  1.0,
  272.6666666666667,
  272.43333333333334,
  0.027643998463948567,
  0.17955415248870848),
 (1.0,
  1.0,
  336.96666666666664,
  336.6666666666667,
  0.041554562250773114,
  0.19999878406524657),
 (1.0,
  1.0,
  421.23333333333335,
  420.8666666666667,
  0.06610753536224365,
  0.32759498755137123),
 (1.0,
  1.0,
  480.96666666666664,
  480.5,
  0.06721206506093343,
  0.44310945669809976),
 (1.0,
  0.9333333333333333,
  540.2857142857143,
  539.7142857142857,
  0.07265985012054443,
  1.3955302323613847)]

In [7]:
# CBS vs ECBS 1.1 random-32x32x10
rand_3 = gen_cmp_list('random-32-32-10.map', 'random-32-32-10-even-1.scen', rand_actors, w=1.1)
rand_3

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

ECBS - Num actors: 5, Rate: 1.00, avg cost: 105.63, time:  0.01
CBS - Num actors: 5, Rate: 1.00, avg cost: 105.63, time:  0.06
Passed in common: 1.0, diff:  1.0000


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

ECBS - Num actors: 23, Rate: 1.00, avg cost: 480.93, time:  0.04
CBS - Num actors: 23, Rate: 1.00, avg cost: 480.50, time:  0.44
Passed in common: 1.0, diff:  1.0009


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

ECBS - Num actors: 26, Rate: 1.00, avg cost: 540.25, time:  0.05
CBS - Num actors: 26, Rate: 0.93, avg cost: 539.71, time:  1.39
Passed in common: 0.9333333333333333, diff:  1.0010


[(1.0,
  1.0,
  105.63333333333334,
  105.63333333333334,
  0.012877957026163737,
  0.05874014695485433),
 (1.0,
  1.0,
  148.86666666666667,
  148.8,
  0.015346988042195638,
  0.061090914408365886),
 (1.0, 1.0, 213.7, 213.6, 0.0168198823928833, 0.0992128849029541),
 (1.0,
  1.0,
  272.6,
  272.43333333333334,
  0.023778255780537924,
  0.17582319577534994),
 (1.0,
  1.0,
  336.8666666666667,
  336.6666666666667,
  0.030270059903462727,
  0.19239864349365235),
 (1.0,
  1.0,
  421.1333333333333,
  420.8666666666667,
  0.03974219163258871,
  0.3273632844289144),
 (1.0,
  1.0,
  480.93333333333334,
  480.5,
  0.04385056495666504,
  0.4357583443323771),
 (1.0,
  0.9333333333333333,
  540.25,
  539.7142857142857,
  0.04531292404447283,
  1.3887967552457536)]